In [1]:
import sys
sys.path.append('/home/lessa/smodels')
from smodels.experiment.databaseObj import Database
from smodels.tools.physicsUnits import GeV,fb,pb
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
db = Database('/home/lessa/smodels-database')

In [3]:
exp = db.getExpResults(analysisIDs=['ATLAS-SUSY-2013-19'],dataTypes=['upperLimit'],txnames=['T2tt'])[0]

In [4]:
tx = exp.getTxNames()[0]
print tx

T2tt


In [5]:
data = tx.txnameData._data

In [12]:
xvals = [pt[0][0][0].asNumber(GeV) for pt in data]
yvals = [pt[0][0][-1].asNumber(GeV) for pt in data]
zvals = [pt[1].asNumber(pb) for pt in data]

In [13]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xvals, yvals, zvals)
ax.set_xlabel('M1')
ax.set_ylabel('m')
ax.set_zlabel('UL')

plt.show()

In [10]:
for ipt,z in enumerate(zvals):
    if z > 40000.:
        print data[ipt]

[[[1.80E+02 [GeV], 0.00E+00 [GeV]], [1.80E+02 [GeV], 0.00E+00 [GeV]]], 5.62E+01 [pb]]
[[[1.90E+02 [GeV], 0.00E+00 [GeV]], [1.90E+02 [GeV], 0.00E+00 [GeV]]], 1.50E+02 [pb]]
[[[2.00E+02 [GeV], 1.00E+01 [GeV]], [2.00E+02 [GeV], 1.00E+01 [GeV]]], 6.49E+01 [pb]]
[[[2.25E+02 [GeV], 3.50E+01 [GeV]], [2.25E+02 [GeV], 3.50E+01 [GeV]]], 5.46E+01 [pb]]
[[[4.00E+02 [GeV], 2.20E+02 [GeV]], [4.00E+02 [GeV], 2.20E+02 [GeV]]], 6.49E+01 [pb]]


In [57]:
m = [[347.406576078*GeV,166.767198233*GeV],[347.406576078*GeV,166.767198233*GeV]]
porig = tx.txnameData.flattenMassArray(m)
pt = ( (np.matrix(porig)[0] - tx.txnameData.delta_x ) ).tolist()[0]
pt = np.dot(pt,tx.txnameData._V)  ## rotate
pt = np.array(pt[:tx.txnameData.dimensionality])
print pt,tx.txnameData.getValueFor(m)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ptm = [pt[0],pt[1],tx.txnameData.getValueFor(m).asNumber(pb)]
ax.scatter([ptm[0]], [ptm[1]], [ptm[2]],c='r',marker='o')
xpts = []
ypts = []
zpts = []
for ipt in tx.txnameData.tri.simplices[tx.txnameData.tri.find_simplex(pt)]:
    xpts.append(tx.txnameData.tri.points[ipt][0])
    ypts.append(tx.txnameData.tri.points[ipt][1])
    zpts.append(tx.txnameData.xsec[ipt])
    print tx.txnameData.tri.points[ipt],tx.txnameData.xsec[ipt]

[ -4.00973477e-10  -1.22160051e+02] 2.30E-01 [pb]
[   5.10991334 -125.03102563] 4.20E+00 
[  -0.17259184 -111.91252194] 1.78E+00 
[-86.89513099 -85.85103305] 4.35E+00 


In [55]:
ax.scatter(xpts, ypts, zpts,c='b',marker='o')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('UL')
plt.show()

In [64]:
fig.clear()
plt.triplot(xpts,ypts)
plt.plot([ptm[0]], [ptm[1]],'ro',xpts,ypts,'bo')
plt.show()

In [67]:
tri = tx.txnameData.tri
simplex = tri.find_simplex([pt])
vertices = np.take(tri.simplices, simplex, axis=0)
temp = np.take(tri.transform, simplex, axis=0)
d=temp.shape[2]
delta = [pt] - temp[:, d]
bary = np.einsum('njk,nk->nj', temp[:, :d, :], delta)
vtx = vertices
wts = np.hstack((bary, 1 - bary.sum(axis=1, keepdims=True)))
v=tx.txnameData.xsec
values = np.array ( [ x.asNumber() for x in tx.txnameData.xsec ] )
ret = np.einsum('nj,nj->n', np.take(values, vtx), wts)
with np.errstate(invalid='ignore'):
    ret[np.any(wts < -1e-10, axis=1)] = np.nan
print ret[0]

4.05666720657


In [69]:
vertices

array([[29, 28, 23]], dtype=int32)

In [70]:
pt

array([ -4.00973477e-10,  -1.22160051e+02])

In [74]:
m = [[347.406576078*GeV,166.767198233*GeV],[347.406576078*GeV,166.767198233*GeV]]
porig = tx.txnameData.flattenMassArray(m)
ptA = ( (np.matrix(porig)[0] - tx.txnameData.delta_x ) ).tolist()[0]
ptA = np.dot(ptA,tx.txnameData._V)  ## rotate
print ptA,tx.txnameData.countNonZeros(ptA)

[ -4.00973477e-10  -1.22160051e+02  -5.68434189e-14   2.62012634e-14] 1
